In [1]:
from analysis.run_evaluation import ingest

In [3]:
ingest()

shape: (0, 0)
┌┐
╞╡
└┘

In [4]:
from analysis.utilities import delete_all_tables

delete_all_tables()

DuckDB tables deleted successfully.
MongoDB collection deleted successfully.


In [2]:
from analysis.run_lobster import pick_one

pick_one()

/Users/venvoo/Library/CloudStorage/GoogleDrive-venvoooo@gmail.com/My Drive/trader/trader_london/analysis/prepare_data.py:49: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  .str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f", strict=False)


                        trading_session_id       Time  Event Type  \
0     ef04f650-ba08-49cb-a0f5-73240fb29979   0.000000           1   
1     27c047b9-37e0-45b1-8c9c-0708434a9184   0.000000           1   
2     ff1d9dcc-230e-47da-9c23-b8600a3c2702   0.000000           1   
3     cb787a30-16f8-42f8-be0a-58d462543cab   0.000000           1   
4     122c5f37-f9bd-43b8-8294-92196bf2cb00   0.000000           1   
...                                    ...        ...         ...   
4101  95317045-d412-4d8f-8820-5b91a409b9f8  59.999772           7   
4102  30eaf597-9ff2-4b36-a2d0-401d11e29b42  59.999823           7   
4103  b1180ade-db12-460d-86f6-f75404181c82  59.999828           1   
4104  b1180ade-db12-460d-86f6-f75404181c82  59.999887           1   
4105  b1180ade-db12-460d-86f6-f75404181c82  59.999947           7   

                      Order ID  Size   Price  Direction  LOBSTER_BOOK_0  \
0     6644c1265393e88cc84a4674   NaN     NaN        NaN             NaN   
1     6644c1265393e88

,trading_session_id,Time,Event Type,Order ID,Size,Price,Direction,LOBSTER_BOOK_0,LOBSTER_BOOK_1,LOBSTER_BOOK_2,...,LOBSTER_BOOK_30,LOBSTER_BOOK_31,LOBSTER_BOOK_32,LOBSTER_BOOK_33,LOBSTER_BOOK_34,LOBSTER_BOOK_35,LOBSTER_BOOK_36,LOBSTER_BOOK_37,LOBSTER_BOOK_38,LOBSTER_BOOK_39
32,3ff39a04-d244-44f1-b9a0-a33465394143,0.000000,1,6644c1265393e88cc84a4693,1.0,2000.0,-1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,3ff39a04-d244-44f1-b9a0-a33465394143,1.999633,1,6644c1285393e88cc84a46af,1.0,2000.0,1.0,2000.0,0.0,2000.0,...,1993.0,0.0,2008.0,0.0,1992.0,0.0,2009.0,0.0,1991.0,0.0
72,3ff39a04-d244-44f1-b9a0-a33465394143,2.999534,1,6644c1295393e88cc84a46bb,1.0,2000.0,1.0,2000.0,0.0,2000.0,...,1993.0,0.0,2008.0,0.0,1992.0,0.0,2009.0,0.0,1991.0,0.0
102,3ff39a04-d244-44f1-b9a0-a33465394143,4.999430,1,6644c12b5393e88cc84a46d9,1.0,1999.0,1.0,2000.0,0.0,2001.0,...,1991.0,0.0,2008.0,0.0,1990.0,0.0,2009.0,0.0,1989.0,0.0
127,3ff39a04-d244-44f1-b9a0-a33465394143,6.000148,1,6644c12c5393e88cc84a46f3,1.0,2001.0,1.0,2000.0,0.0,2000.0,...,1990.0,0.0,2008.0,0.0,1989.0,0.0,2009.0,0.0,1988.0,0.0
161,3ff39a04-d244-44f1-b9a0-a33465394143,9.000304,1,6644c12f5393e88cc84a4715,1.0,1995.0,1.0,2001.0,1.0,1999.0,...,1989.0,0.0,2009.0,0.0,1988.0,0.0,2010.0,0.0,1987.0,0.0
185,3ff39a04-d244-44f1-b9a0-a33465394143,10.999914,1,6644c1315393e88cc84a472d,NaN,NaN,NaN,2001.0,1.0,1999.0,...,1989.0,0.0,2009.0,0.0,1988.0,0.0,2010.0,0.0,1987.0,0.0
224,3ff39a04-d244-44f1-b9a0-a33465394143,14.000060,1,6644c1345393e88cc84a4753,1.0,2001.0,-1.0,2001.0,1.0,1999.0,...,1989.0,0.0,2009.0,0.0,1988.0,0.0,2010.0,0.0,1987.0,0.0
253,3ff39a04-d244-44f1-b9a0-a33465394143,15.999879,1,6644c1365393e88cc84a4771,1.0,2006.0,-1.0,2001.0,1.0,1999.0,...,1989.0,0.0,2009.0,0.0,1988.0,0.0,2010.0,0.0,1987.0,0.0
279,3ff39a04-d244-44f1-b9a0-a33465394143,17.000366,1,6644c1375393e88cc84a478a,1.0,1998.0,-1.0,2001.0,1.0,1999.0,...,1989.0,0.0,2009.0,0.0,1988.0,0.0,2010.0,0.0,1987.0,0.0


In [5]:
remote.columns

['id',
 'trading_session_id',
 'content',
 'text',
 'type',
 'order_book',
 'active_orders',
 'history',
 'spread',
 'midpoint',
 'transaction_price',
 'timestamp',
 'order_amount',
 'order_price',
 'order_type',
 'order_trader_id']